In [1]:
import sys
import pandas as pd
import numpy as np
from typing import Any
from scipy.stats import linregress, ttest_1samp

%load_ext autoreload
%autoreload 2
# sys.path.append("D:/Github/note/module")                        # for windows
sys.path.append("/Users/xinc./Documents/GitHub/note/module")    # for mac
from get_info_FinMind import GetInfoFinMind
FinMind_fetcher = GetInfoFinMind()
from plot_func import plot, plot_scatter, plot_df_columns, plot_pdf, plot_dropped_positions, plot_sequence
from performance_func import summarize_performance, mean_ttest

start_date = pd.to_datetime("2020-01-01", format = "%Y-%m-%d")
end_date = pd.to_datetime("2025-09-25", format = "%Y-%m-%d")


# download data

In [157]:
# TXO
op_df = FinMind_fetcher.get_option_daily("TXO", start_date, end_date, trading_session = "all")
op_df.reset_index(drop = True, inplace = True)
op_df_position = op_df[op_df["trading_session"] == "position"].copy()

# TXOa
op_df_after = op_df[op_df["trading_session"] == "after_market"].copy()
op_df_position.to_pickle("../data/TXO.pkl")
op_df_after.to_pickle("../data/TXO_after.pkl")

# TX
tx_df = FinMind_fetcher.init_df("TX", start_date, end_date)
tx_df = FinMind_fetcher.get_future_price()
tx_df.to_csv("../data/TX.csv")

# TXa
txa_df = FinMind_fetcher.init_df("TX", start_date, end_date)
txa_df = FinMind_fetcher.get_future_price(trading_session = "after_market")
txa_df.to_csv("../data/TX_after.csv")


2025-10-21 11:10:47.150 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanOptionDaily, data_id: TXO
2025-10-21 11:12:26.065 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanFuturesDaily, data_id: TX
/Users/xinc./Documents/GitHub/note/module/get_info_FinMind.py:267: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

2025-10-21 11:12:28.603 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanFuturesDaily, data_id: TX
/Users/xinc./Documents/GitHub/note/module/get_info_FinMind.py:267: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping col

# read

In [2]:
"""read tx, txo"""
tx_df = pd.read_csv("../data/TX.csv")
txa_df = pd.read_csv("../data/TX_after.csv")
op_df = pd.read_pickle("../data/TXO.pkl")
opa_df = pd.read_pickle("../data/TXO_after.pkl")
twii_df = pd.read_pickle("../data/twii.pkl")
twii_df.index = pd.to_datetime(twii_df.index)
twii_df = twii_df.loc[start_date: end_date]

for frame in (tx_df, txa_df):
    frame["Timestamp"] = pd.to_datetime(frame["Timestamp"])
op_df["date"] = pd.to_datetime(op_df["date"])
opa_df["date"] = pd.to_datetime(opa_df["date"])

def build_atm(option_df: pd.DataFrame, price_map: pd.Series, label: str) -> pd.DataFrame:
    atm = option_df.copy()
    atm[label] = atm["date"].map(price_map)
    atm = atm[atm[label].notna()].copy()
    atm["moneyness_gap"] = (atm["strike_price"] - atm[label]).abs()
    atm = (atm.sort_values(["date", "call_put", "moneyness_gap", "volume", "open_interest"],
                          ascending = [True, True, True, False, False])
              .groupby(["date", "call_put"], as_index = False)
              .head(1)
              .reset_index(drop = True))
    return atm.drop(columns = [label, "moneyness_gap"])

_tx_close_map = tx_df.set_index("Timestamp")["Close"]
_tx_open_map = tx_df.set_index("Timestamp")["Open"]
op_atm_close_df = build_atm(op_df, _tx_close_map, "day_close")
op_atm_open_df = build_atm(op_df, _tx_open_map, "day_open")

_txa_close_map = txa_df.set_index("Timestamp")["Close"]
_txa_open_map = txa_df.set_index("Timestamp")["Open"]
opa_atm_close_df = build_atm(opa_df, _txa_close_map, "night_close")
opa_atm_open_df = build_atm(opa_df, _txa_open_map, "night_open")

# diff scenario

## 夜盤 close - 日盤 open

### put

#### ret

In [133]:
put_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", ["date", "strike_price", "close"]].rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df = put_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
put_mask = (put_df["prev_night_close"] > 0) & (put_df["open"] > 0)
put_df["cross_ret"] = np.where(put_mask, (put_df["open"] / put_df["prev_night_close"]) - 1, np.nan)

# calculate date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] / merge_df["Close"]) - 1
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])

put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [134]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

# normalize cum ret
put_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_cross_ret"] / put_df["cum_demeaned_cross_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [107]:
put_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", ["date", "strike_price", "close"]].rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df = put_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
put_mask = (put_df["prev_night_close"] > 0) & (put_df["open"] > 0)
put_df["cross_ret"] = np.where(put_mask, (put_df["open"] - put_df["prev_night_close"]), np.nan)

# calculate date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] - merge_df["Close"])
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])

put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [108]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum cross ret
put_df["cum_cross_ret"] = put_df["cross_ret"].cumsum()
put_df.loc[mask, "cum_cross_ret"] = put_df.loc[mask, "cross_ret"].cumsum()
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()
put_df.loc[mask, "cum_demeaned_cross_ret"] = put_df.loc[mask, "demeaned_cross_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()
put_df.loc[mask, "cum_demeaned_tx_ret"] = put_df.loc[mask, "demeaned_tx_ret"].cumsum()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [131]:
call_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", ["date", "strike_price", "close"]]
              .rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_close"] > 0) & (call_df["open"] > 0)
call_df["cross_ret"] = np.where(call_mask, (call_df["open"] / call_df["prev_night_close"]) - 1, np.nan)
call_df["gap_days"] = call_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] / merge_df["Close"]) - 1
call_df["date"], merge_df["Timestamp"] = pd.to_datetime(call_df["date"]), pd.to_datetime(merge_df["Timestamp"])

call_df = call_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [132]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
mean = call_df["cross_ret"].mean()
call_df["demeaned_cross_ret"] = call_df["cross_ret"] - mean
call_df["cum_demeaned_cross_ret"] = call_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

# normalize
call_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_cross_ret"] / call_df["cum_demeaned_cross_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [85]:
call_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", "date"].unique())

# get cross ret
call_df = call_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", ["date", "strike_price", "close"]]
            .rename(columns = {"close": "prev_night_close"}),
            left_index = True,
            right_on = "date",
            how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_close"] > 0) & (call_df["open"] > 0)
call_df["cross_ret"] = np.where(call_mask, (call_df["open"] - call_df["prev_night_close"]), np.nan)
call_df["gap_days"] = call_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] - merge_df["Close"])
call_df["date"], merge_df["Timestamp"] = pd.to_datetime(call_df["date"]), pd.to_datetime(merge_df["Timestamp"])

call_df = call_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [86]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum cross ret
call_df["cum_cross_ret"] = call_df["cross_ret"].cumsum()
call_df.loc[mask, "cum_cross_ret"] = call_df.loc[mask, "cross_ret"].cumsum()
mean = call_df["cross_ret"].mean()
call_df["demeaned_cross_ret"] = call_df["cross_ret"] - mean
call_df["cum_demeaned_cross_ret"] = call_df["demeaned_cross_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_cross_ret"] = call_df.loc[mask, "demeaned_cross_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_tx_ret"] = call_df.loc[mask, "demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] < 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

## 夜盤 open - 日盤 open

### put

#### ret

In [129]:
put_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open"]]
              .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)

put_df["date"] = pd.to_datetime(put_df["date"])
position_open = (
    op_df.loc[(op_df["call_put"] == "put"),
              ["date", "strike_price", "open"]]
        .drop_duplicates(["date", "strike_price"])
        .rename(columns = {"open": "day_open"})
)
put_df = put_df.merge(position_open, on = ["date", "strike_price"], how = "left")
mask = (put_df["prev_night_open"] > 0) & (put_df["day_open"] > 0)
put_df["cross_ret"] = np.where(mask, (put_df["day_open"] / put_df["prev_night_open"]) - 1, np.nan)

# get date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}), txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["position_open"] / merge_df["after_open"]) - 1
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])
put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [130]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross_ret ret
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

# normalize
put_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_cross_ret"] / put_df["cum_demeaned_cross_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [115]:
put_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open"]]
                .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)

put_df["date"] = pd.to_datetime(put_df["date"])
position_open = (
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]]
    .drop_duplicates(["date", "strike_price"])
    .rename(columns = {"open": "day_open"})
)
put_df = put_df.merge(position_open, on = ["date", "strike_price"], how = "left")
mask = (put_df["prev_night_open"] > 0) & (put_df["day_open"] > 0)
put_df["cross_ret"] = np.where(mask, (put_df["day_open"] - put_df["prev_night_open"]), np.nan)

# get date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}), txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["position_open"] - merge_df["after_open"])
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])
put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [116]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum cross_ret ret
put_df["cum_cross_ret"] = put_df["cross_ret"].cumsum()
put_df.loc[mask, "cum_cross_ret"] = put_df.loc[mask, "cross_ret"].cumsum()
mean = put_df["cross_ret"].mean()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [127]:
call_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open"]]
               .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]]
          .rename(columns = {"open": "day_open"}),
    on = ["date", "strike_price"],
    how = "left"
)
call_open_mask = (call_df["prev_night_open"] > 0) & (call_df["day_open"] > 0)
call_df["ret"] = np.where(
    call_open_mask,
    (call_df["day_open"] / call_df["prev_night_open"]) - 1,
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}),
    txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["Timestamp"] = pd.to_datetime(merge_df["Timestamp"])
merge_df["tx_ret"] = (merge_df["position_open"] / merge_df["after_open"]) - 1
call_df = call_df.merge(
    merge_df.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["ret", "tx_ret"], inplace = True)

In [128]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = call_df["ret"].mean()
call_df["demeaned_ret"] = call_df["ret"] - mean
call_df["cum_demeaned_ret"] = call_df["demeaned_ret"].cumsum()

mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

call_df[["norm_cum_demeaned_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_ret"] / call_df["cum_demeaned_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [109]:
call_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open"]]
               .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]]
          .rename(columns = {"open": "day_open"}),
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_open"] > 0) & (call_df["day_open"] > 0)
call_df["ret"] = np.where(
    call_mask,
    (call_df["day_open"] - call_df["prev_night_open"]),
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}),
    txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["Timestamp"] = pd.to_datetime(merge_df["Timestamp"])
merge_df["tx_ret"] = (merge_df["position_open"] - merge_df["after_open"])
call_df = call_df.merge(
    merge_df.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["ret", "tx_ret"], inplace = True)

In [110]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum open_to_open ret
call_df["cum_ret"] = call_df["ret"].cumsum()
call_df.loc[mask, "cum_ret"] = call_df.loc[mask, "ret"].cumsum()
mean = call_df["ret"].mean()
call_df["demeaned_ret"] = call_df["ret"] - mean
call_df["cum_demeaned_ret"] = call_df["demeaned_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_ret"] = call_df.loc[mask, "demeaned_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_tx_ret"] = call_df.loc[mask, "demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")

## 放假前 日盤 open - 日盤 close

### put

#### ret

In [125]:
"""get date gap days"""
put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy().reset_index(drop = True)

put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df["gap_days"] = put_df["gap_days"].shift(-1)
put_df["daily_ret"] = (put_df["close"] / put_df["open"]) - 1

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] / tx_day_ret["Open"]) - 1
put_df = put_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [126]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = put_df["daily_ret"].mean()
put_df["demeaned_daily_ret"] = put_df["daily_ret"] - mean
put_df["cum_demeaned_daily_ret"] = put_df["demeaned_daily_ret"].cumsum()

mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

put_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_daily_ret"] / put_df["cum_demeaned_daily_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")

#### point

In [3]:
"""get date gap days"""
put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy().reset_index(drop = True)

put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df["gap_days"] = put_df["gap_days"].shift(-1)
put_df["ret"] = (put_df["close"] - put_df["open"])

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] - tx_day_ret["Open"])
put_df = put_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["ret", "tx_ret"], inplace = True)

In [5]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum daily ret
put_df["cum_ret"] = put_df["ret"].cumsum()
put_df.loc[mask, "cum_ret"] = put_df.loc[mask, "ret"].cumsum()
mean = put_df["ret"].mean()
put_df["demeaned_ret"] = put_df["ret"] - mean
put_df["cum_demeaned_ret"] = put_df["demeaned_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [9]:
"""get date gap days"""
call_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy().reset_index(drop = True)

call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df["gap_days"] = call_df["gap_days"].shift(-1)
call_df["daily_ret"] = (call_df["close"] / call_df["open"]) - 1

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] / tx_day_ret["Open"]) - 1
call_df = call_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [10]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

call_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_daily_ret"] / call_df["cum_demeaned_daily_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")

#### point

In [11]:
call_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy().reset_index(drop = True)

call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df["gap_days"] = call_df["gap_days"].shift(-1)
call_df["daily_ret"] = (call_df["close"] - call_df["open"])

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] - tx_day_ret["Open"])
call_df = call_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [12]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum daily ret
call_df["cum_daily_ret"] = call_df["daily_ret"].cumsum()
call_df.loc[mask, "cum_daily_ret"] = call_df.loc[mask, "daily_ret"].cumsum()
mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")

### compare to index

In [17]:
tw_df = twii_df.copy()
tw_df["daily_ret"] = (tw_df["close"] - tw_df["open"])

put_df.set_index("date", inplace = True)
call_df.set_index("date", inplace = True)
put_df.index, call_df.index, tw_df.index = pd.to_datetime(put_df.index), pd.to_datetime(call_df.index), pd.to_datetime(tw_df.index)

tw_df = tw_df.reindex(put_df.index)
tw_df["cum_daily_ret"] = tw_df["daily_ret"].cumsum()

put_df.reset_index(inplace = True)
call_df.reset_index(inplace = True)
tw_df.reset_index(inplace = True)

ret_df = pd.DataFrame({
    "cum_tw_ret": tw_df["cum_daily_ret"],
    "cum_put_ret": put_df["cum_ret"],
    "cum_call_ret": call_df["cum_daily_ret"]
})

ret_df_norm = ret_df / ret_df.std()
plot_df_columns(ret_df_norm, ["cum_tw_ret", "cum_put_ret", "cum_call_ret"])


## 放假前 夜盤 open - 夜盤 close

### put

#### ret

In [23]:
put_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open", "close"]]
    .copy()
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_mask = (put_df["open"] > 0) & (put_df["close"] > 0)
put_df["daily_ret"] = np.where(
    put_mask,
    (put_df["close"] / put_df["open"]) - 1,
    np.nan
)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] / txa_day_ret["Open"]) - 1
put_df = put_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [24]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = put_df["daily_ret"].mean()
put_df["demeaned_daily_ret"] = put_df["daily_ret"] - mean
put_df["cum_demeaned_daily_ret"] = put_df["demeaned_daily_ret"].cumsum()

mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

put_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_daily_ret"] / put_df["cum_demeaned_daily_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [26]:
put_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open", "close"]]
    .copy()
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_mask = (put_df["open"] > 0) & (put_df["close"] > 0)
put_df["daily_ret"] = np.where(
    put_mask,
    (put_df["close"] - put_df["open"]),
    np.nan
)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] - txa_day_ret["Open"])
put_df = put_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [27]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum daily ret
put_df["cum_daily_ret"] = put_df["daily_ret"].cumsum()
put_df.loc[mask, "cum_daily_ret"] = put_df.loc[mask, "daily_ret"].cumsum()
mean = put_df["daily_ret"].mean()
put_df["demeaned_daily_ret"] = put_df["daily_ret"] - mean
put_df["cum_demeaned_daily_ret"] = put_df["demeaned_daily_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [28]:
call_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open", "close"]]
    .copy()
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_mask = (call_df["open"] > 0) & (call_df["close"] > 0)
call_df["daily_ret"] = np.where(
    call_mask,
    (call_df["close"] / call_df["open"]) - 1,
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] / txa_day_ret["Open"]) - 1
call_df = call_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [29]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

call_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_daily_ret"] / call_df["cum_demeaned_daily_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [35]:
call_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open", "close"]]
    .copy()
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_mask = (call_df["open"] > 0) & (call_df["close"] > 0)
call_df["daily_ret"] = np.where(
    call_mask,
    (call_df["close"] - call_df["open"]),
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] - txa_day_ret["Open"])
call_df = call_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [36]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum daily ret
call_df["cum_daily_ret"] = call_df["daily_ret"].cumsum()
call_df.loc[mask, "cum_daily_ret"] = call_df.loc[mask, "daily_ret"].cumsum()
mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")

# backtest

## benchmark

In [51]:
short_call_df = op_atm_df.loc[op_atm_df["call_put"] == "call"].copy()


short_call_df


,date,option_id,contract_date,strike_price,call_put,open,max,min,close,volume,settlement_price,open_interest,trading_session
0,2020-01-02,TXO,202001W1,12100.0,call,5.7,21.0,1.2,2.0,134030,0.0,16210,position
2,2020-01-03,TXO,202001W2,12100.0,call,109.0,122.0,31.5,52.0,52247,52.0,8766,position
4,2020-01-06,TXO,202001W2,11950.0,call,119.0,119.0,47.0,47.5,16541,48.5,4983,position
6,2020-01-07,TXO,202001W2,11850.0,call,150.0,165.0,28.0,46.0,55996,46.0,8700,position
8,2020-01-08,TXO,202001W2,11800.0,call,15.0,98.0,12.0,39.5,121111,0.0,3741,position
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780,2025-09-19,TXO,202509W4,25650.0,call,276.0,276.0,158.0,167.0,1440,167.0,539,position
2782,2025-09-22,TXO,202509W4,25900.0,call,63.0,128.0,40.0,99.0,18975,99.0,3500,position
2784,2025-09-23,TXO,202509W4,26250.0,call,25.0,136.0,17.5,86.0,21424,86.0,2751,position
2786,2025-09-24,TXO,202509W4,26250.0,call,136.0,143.0,0.1,0.2,42946,0.0,6890,position


賣方會卡保證金，所以utility會少一點，所以要考慮保證金使用的比例
看call的，報酬金額 / 保證金（市值）

先測整段連續再分放假